In [1]:
import os
from circuitexplorer import CircuitExplorer
from bioexplorer import BioExplorer, Metabolism
%matplotlib notebook

URL = 'localhost:5000'

be = BioExplorer(URL)
core = be.core_api()
be.reset_scene()

ce = CircuitExplorer(core)
data_folder = os.getenv('METABOLISM_DATA_FOLDER')

In [2]:
load_neuron = True
load_other_neurons = True
load_astrocyte = False
load_vasculature = False
use_sdf = True
generate_movie = False

In [3]:
if load_neuron:
    print('Loading neuron')
    config = '/gpfs/bbp.cscs.ch/project/proj129/medias/circuits/o1/CircuitConfig'
    neuron = ce.load_circuit(
        use_sdf_soma=True, use_sdf_branches=True, 
        use_sdf_nucleus=True, use_sdf_mitochonria=True,
        use_sdf_synapses=True,
        load_afferent_synapses=True, load_efferent_synapses=True,
        load_soma=True, load_axon=True, 
        load_apical_dendrite=False, load_dendrite=False,
        circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NONE,
        morphology_color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_NONE,
        morphology_quality=ce.GEOMETRY_QUALITY_MEDIUM,
        generate_internals=True,
        path=config, gids=[170234 + 1]
    )
    neuron_model_id = neuron['id']

    if load_other_neurons:
        ids = list()
        for i in range(100):
            ids.append(170234 + i * 50)

        other_neurons = ce.load_circuit(
            use_sdf_soma=True, use_sdf_branches=False,
            use_sdf_nucleus=True,
            use_sdf_mitochonria=True,
            load_afferent_synapses=False, load_efferent_synapses=False,
            load_soma=True, load_axon=True, 
            load_apical_dendrite=True, load_dendrite=True,
            circuit_color_scheme=ce.CIRCUIT_COLOR_SCHEME_NONE,
            morphology_color_scheme=ce.MORPHOLOGY_COLOR_SCHEME_NONE,
            morphology_quality=ce.GEOMETRY_QUALITY_MEDIUM,
            generate_internals=False,
            path=config, gids=ids
        )
        other_neurons_id = other_neurons['id']

if load_astrocyte:
    print('Loading astrocyte...')
    config = '/gpfs/bbp.cscs.ch/project/proj129/medias/metabolism/astrocytes/one.astrocytes'
    astrocytes = ce.load_astrocytes(
        path=config, generate_internals=True, use_sdf_soma=use_sdf)
    astrocyte_model_id = astrocytes['id']

if load_vasculature:
    print('Loading vasculature...')
    config = '/gpfs/bbp.cscs.ch/project/proj129/caches/neuroscience/ngv/vasculature.brayns'
    vasculature = core.add_model(path=config, name='Vasculature')
    vasculature_model_id = vasculature['id']


Loading neuron


RequestError: Unsupported type

In [ ]:
def set_morphology_materials(model_id, opacity, color1, color2=[1,1,1], color3=[1,0,1]):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    emissions = list()
    reflection_indices = list()
    
    material_ids = ce.get_material_ids(model_id)['ids']
    for material_id in material_ids:
        mid = material_id % ce.NB_MATERIALS_PER_MORPHOLOGY
        if mid in [ce.MATERIAL_OFFSET_AFFERENT_SYNPASE, ce.MATERIAL_OFFSET_EFFERENT_SYNPASE]:
            opacities.append(1.0)
            colors.append(color2)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(1.0)
            specular_exponents.append(3.0)
            emissions.append(0.75)
            reflection_indices.append(0.0)
        elif mid == ce.MATERIAL_OFFSET_MITOCHONDRION:
            opacities.append(1.0)
            colors.append(color3)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            glossinesses.append(0.5)
            user_params.append(1.0)
            specular_exponents.append(6.0)
            emissions.append(0.25)
            reflection_indices.append(0.0)
        elif mid == ce.MATERIAL_OFFSET_NUCLEUS:
            opacities.append(1.0)
            colors.append([0.9, 0.9, 0.9])
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            glossinesses.append(1.0)
            user_params.append(0.2)
            specular_exponents.append(30.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        elif mid == ce.MATERIAL_OFFSET_SOMA:
            opacities.append(1.0)
            colors.append(color1)
            shading_modes.append(ce.SHADING_MODE_ELECTRON_TRANSPARENCY)
            glossinesses.append(1.0)
            user_params.append(2.0)
            specular_exponents.append(50.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
        else:
            # Membrane
            opacities.append(opacity)
            colors.append(color1)
            shading_modes.append(ce.SHADING_MODE_PERLIN)
            glossinesses.append(0.7)
            user_params.append(0.02)
            specular_exponents.append(50.0)
            emissions.append(0.0)
            reflection_indices.append(0.0)
            
        refraction_indices.append(1.2)
        
    ce.set_material_extra_attributes(model_id)
    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        reflection_indices=reflection_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses,
        emissions=emissions
    )
    
def set_vasculature_materials(model_id):
    colors = list()
    opacities = list()
    refraction_indices = list()
    specular_exponents = list()
    shading_modes = list()
    user_params = list()
    glossinesses = list()
    
    material_ids = ce.get_material_ids(model_id)['ids']
    for _ in material_ids:
        opacities.append(0.75)
        colors.append([1, 0, 0])
        shading_modes.append(ce.SHADING_MODE_PERLIN)
        glossinesses.append(0.9)
        user_params.append(0.01)
        specular_exponents.append(3.0)
        refraction_indices.append(2.5)
        
    ce.set_material_extra_attributes(model_id)
    ce.set_materials(
        model_ids=[model_id], material_ids=material_ids,
        diffuse_colors=colors, specular_colors=colors,
        opacities=opacities, refraction_indices=refraction_indices,
        shading_modes=shading_modes, specular_exponents=specular_exponents,
        user_parameters=user_params, glossinesses=glossinesses
    )
    
if load_neuron:
    set_morphology_materials(neuron_model_id, 0.5, [1.0,1.0,0.8], [0.5,1,0.5], [0.25,0.2,0.15])
if load_other_neurons:
    set_morphology_materials(other_neurons_id, 1.0, [1.0,1.0,0.8], [0.5,1,0.5], [1,0,1])
if load_astrocyte:
    set_morphology_materials(astrocyte_model_id, 1.0, [1,1,0], [1,1,1], [1,0,1])
if load_vasculature:
    set_vasculature_materials(vasculature_model_id)
status = core.set_renderer()

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
mm.get_camera()

In [ ]:
mm.set_camera(
    direction=[0.0, 0.0, -1.0],
    origin=[457.1072774501307, 706.3183526678731, 869.0934655148297],
    up=[0.0, 1.0, 0.0]
)

In [ ]:
camera_key_frames=[
{'apertureRadius': 0.0,
 'direction': [0.49360470041940435, -0.47370643244327776, 0.72935356006928],
 'focusDistance': 1000000.0,
 'origin': [461.239738161088, 763.8795171406866, 855.0383064429964],
 'up': [0.46258708982009183, 0.8531724175524209, 0.24106017974675104]},
{'apertureRadius': 0.0,
 'direction': [0.662736008065918, -0.686988035651692, 0.2980409745056567],
 'focusDistance': 1000000.0,
 'origin': [461.0893755332648, 760.0200258975466, 851.3944538111009],
 'up': [0.6187635148103309, 0.7265415447417015, 0.29877934417209573]},
{'apertureRadius': 0.0,
 'direction': [-0.026775866281874006,
  -0.9885431834110916,
  0.14854436211556554],
 'focusDistance': 1000000.0,
 'origin': [465.5099208287645, 753.7016373950581, 853.6238629921304],
 'up': [0.7188963577153573, 0.08421095435439807, 0.6899974942203158]},
{'apertureRadius': 0.0,
 'direction': [-0.18081358183570112,
  -0.9776176717214384,
  -0.10756456926747726],
 'focusDistance': 1000000.0,
 'origin': [460.99289145371966, 740.046945659778, 852.4836508488913],
 'up': [0.7047324180955374, -0.20507185495847113, 0.6791890408340733]},
{'apertureRadius': 0.0,
 'direction': [-0.15558109096705808, -0.9849732822664798, 0.07498104663644978],
 'focusDistance': 1000000.0,
 'origin': [459.5440003316537, 726.0698097752108, 852.4256802744735],
 'up': [0.7227761026837128, -0.06176777853112814, 0.6883163857737751]},
{'apertureRadius': 0.0,
 'direction': [-0.12111618330322638, -0.9835108051409053, 0.1343032624813998],
 'focusDistance': 1000000.0,
 'origin': [458.3785605564927, 709.9279101267985, 854.749463954448],
 'up': [0.7296963884038286, 0.003511362534427853, 0.6837622767333403]},    
{'apertureRadius': 0.0,
 'direction': [0.1603460928829913, 0.11763301716037507, -0.9800263281008863],
 'focusDistance': 1000000.0,
 'origin': [456.84763123982674, 706.0758364777412, 863.6674212421925],
 'up': [0.9797721550430045, 0.10146850102262095, 0.17248381808914465]}
]

In [ ]:
mm.build_camera_path(camera_key_frames, 50, 25)

In [ ]:
status = core.set_renderer(current='circuit_explorer_advanced',
    subsampling=4, max_accum_frames=64)
params = core.CircuitExplorerAdvancedRendererParams()
params.exposure = 1.2
params.max_bounces = 10
params.epsilon_factor = 500.0
params.fog_start = 40.0
params.fog_thickness = 250.0
params.gi_samples = 1
params.gi_weight = 0.2
params.gi_distance = 3.0
params.shadows = 1.0
params.soft_shadows = 1.0
status = core.set_renderer_params(params)

In [ ]:
if generate_movie:
    for frame in range(mm.get_nb_frames()):
        mm.set_current_frame(frame)
        mm.create_snapshot(
            renderer='circuit_explorer_advanced',
            size=[960,540], samples_per_pixel=32,
            path='/gpfs/bbp.cscs.ch/project/proj129/movies/neuromodulation/perspective/960x540/circuit_explorer_advanced',
            base_name='%05d' % frame
        )
else:
    import time
    for frame in range(0, mm.get_nb_frames(), 2):
        mm.set_current_frame(frame)
        time.sleep(0.2)

In [ ]:
mm.set_current_frame(300)

In [ ]:
from bioexplorer import Widgets
w = Widgets(be)

In [ ]:
w.display_focal_distance()